In [1]:
import pandas as pd
import numpy as np 
import datetime
import os

def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
start_date_2019Q4=datetime.date(2019,11,3)
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)


In [2]:
# POS date
file_POS_weeks_1="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt"
file_POS_weeks_2="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-16/MediaStormDailySales20191119-112232-478.txt"
file_POS_weeks_3="/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-23/MediaStormDailySales20191126-112901-552.txt"
file_POS_weeks_4="/home/jian/BigLots/MediaStorm_2019-11-30/MediaStormDailySales20191203.txt"

list_all_files=[file_POS_weeks_1,file_POS_weeks_2,file_POS_weeks_3,file_POS_weeks_4]

In [4]:
df_all_POS_by_store=pd.DataFrame()
for file in list_all_files:
    df=pd.read_csv(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    
    df_rewards_sales=df_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"rewards_sales"})
    df_rewards_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans['rewards_trans']=1
    df_rewards_trans=df_rewards_trans.groupby(['location_id','transaction_dt'])['rewards_trans'].sum().to_frame().reset_index()
    df_rewards=pd.merge(df_rewards_sales,df_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df_non_rewards_sales=df_non_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"non_rewards_sales"})
    df_non_rewards_trans=df_non_rewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_non_rewards_trans['non_rewards_trans']=1
    df_non_rewards_trans=df_non_rewards_trans.groupby(['location_id','transaction_dt'])['non_rewards_trans'].sum().to_frame().reset_index()
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df=pd.merge(df_rewards,df_non_rewards,on=['location_id','transaction_dt'],how="outer")
    df['week_end_dt']=df['transaction_dt'].max()
    df_all_POS_by_store=df_all_POS_by_store.append(df)
    print(datetime.datetime.now(),df['transaction_dt'].max())


2019-12-05 11:16:32.887667 2019-11-09
2019-12-05 11:17:27.832268 2019-11-16
2019-12-05 11:18:29.585986 2019-11-23
2019-12-05 11:19:52.778638 2019-11-30


In [8]:
df_all_POS_by_store['store_type']=np.where(df_all_POS_by_store['location_id']=="6990","online","instore")
df_summary=df_all_POS_by_store.groupby(["week_end_dt",'store_type'])['rewards_sales','non_rewards_sales','rewards_trans','non_rewards_trans'].sum().reset_index()

In [9]:
writer=pd.ExcelWriter("./BL_first_4_weeks_sales_2019Q4_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary.to_excel(writer,"df_summary",index=False)
df_all_POS_by_store.to_excel(writer,"df_by_store_date",index=False)
writer.save()

In [10]:
df_all_POS_by_store.shape

(39740, 8)

In [11]:
df_all_POS_by_store.head(3)

,location_id,transaction_dt,rewards_sales,rewards_trans,non_rewards_sales,non_rewards_trans,week_end_dt,store_type
0,1,2019-11-03,11861.60,213,3845.84,154,2019-11-09,instore
1,1,2019-11-04,4648.47,147,2840.52,96,2019-11-09,instore
2,1,2019-11-05,4513.96,120,2211.68,127,2019-11-09,instore
